In [45]:
from flask import Flask , render_template, request
from werkzeug.utils import secure_filename
import os
import pymongo as pm

In [44]:
conn = pm.MongoClient('localhost:27017')
dname = 'mydb'
db = conn.get_database(name)
def insert_data(db_name, args_dict):
    '''
    db_name -> string i.e name of the db
    args_dict -> a dictionary of entries in db
    '''
    db[db_name].insert_one(args_dict)
    print('Data inserted successfully')

def read_data(db_name):
    '''
    returns a cursor of objects
    which can be iterated and printed
    '''
    cols = db[db_name].find({})
    return cols

#Update in data base
def update_data(db_name, idno, updation):
    '''
    db_name -> string
    idno -> id number of database entry in dict
    eg:- {'id':'02'}
    updation -> dict of elements to be updated
    eg:-{
        '$set':{
            'name':'Kevin11',
            'contact':'9664820165'
        }
    }
    '''
    db[db_name].update_one(idno, updation)
    print('Database updated successfully')

def delete_row(db_name, idno):
    '''
    Deletes the complete row 
    idno must be a dict {idno:'anything'}
    '''
    db[db_name].delete_many(idno)
    print('Row deleted')

In [47]:
app = Flask(__name__)
app.config['UPLOAD_FOLDER'] = './uploads'
filters = ['jpg', 'pdf', 'jpeg', 'png', 'jfif', 'rar', 'zip']
img_path = './uploads'
@app.route('/')
def upload_file():
    return render_template('upload.html')

@app.route('/uploaded', methods=['GET', 'POST'])
def uploader():
    if request.method == 'POST':
        file = request.files['file']
        print(secure_filename(file.filename))
        name = secure_filename(file.filename)
        if name.split('.')[1] in filters:
            file.save(os.path.join(app.config['UPLOAD_FOLDER'],secure_filename(file.filename)))
            args = {'name':name,
                    'path':os.path.join(app.config['UPLOAD_FOLDER'],secure_filename(file.filename))}
            insert_data(dname, args_dict=args)
            return 'file uploaded successfully'
        else:
            return 'format is not supported'

In [48]:
app.run()

 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [14/Mar/2018 12:16:22] "GET / HTTP/1.1" 200 -


test.zip


127.0.0.1 - - [14/Mar/2018 12:16:37] "POST /uploaded HTTP/1.1" 200 -


Data inserted successfully


In [52]:
import cv2
import numpy as np
def read_image(img_name, dbname):
    '''
    Get image path form mongodb and display function
    img_name -> string
    dbname -> mongodb name in string
    '''
    cols = read_data(db_name=dbname)
    for c in cols:
        if c['name'] == img_name:
            img = cv2.imread(c['path'])
            cv2.imshow('dbimg', img)
            cv2.waitKey(0)
            cv2.destroyAllWindows()
            
read_image(img_name='cpyuigoku.jpg', dbname=dname)